In [1]:
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [5]:
import cv2
import numpy as np
import tensorflow as tf
from pathlib import Path

2026-02-16 13:46:56.332784: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-16 13:46:56.432240: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-16 13:46:58.281828: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
from graph import EpilepsyState

In [7]:
PROJECT_ROOT

PosixPath('/home/aasritha/EpilepsyNexus')

In [86]:
MODEL_PATH = PROJECT_ROOT / "models" / "efficientnet_v2_finetuned.h5"
model = tf.keras.models.load_model(MODEL_PATH)

In [36]:
from tensorflow.keras.preprocessing import image


In [73]:
def crop_brain(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    if len(contours) > 0:
        cnt = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        image = image[y:y+h, x:x+w]

    return image


def resize_image(image, size=224):
    return cv2.resize(image, (size, size))


def preprocess_mri_inference(image):
    """
    EXACT match with your saved training dataset.
    """

    # ---- crop ----
    image = crop_brain(image)

    # ---- resize ----
    image = resize_image(image)

    # ---- same min-max scaling ----
    image = image.astype(np.float32)
    image = (image - image.min()) / (image.max() - image.min() + 1e-8)

    # ---- convert to uint8 (same as saved dataset) ----
    image = (image * 255).astype(np.uint8)

    # ---- final normalization ----
    image = image.astype(np.float32) / 255.0

    return image


# =====================================================
# 3️⃣ MRI NODE
# =====================================================

def mri_classifier_node(state: EpilepsyState) -> EpilepsyState:
    """
    MRI epilepsy classification.
    """

    if state.mri_image_path is None:
        state.mri_epilepsy_label = "uncertain"
        return state

    try:
        # ---- load image ----
        img = cv2.imread(state.mri_image_path)

        if img is None:
            state.mri_epilepsy_label = "uncertain"
            return state

        # ---- preprocess ----
        processed = preprocess_mri_inference(img)

        # ---- add batch ----
        processed = np.expand_dims(processed, axis=0)

        # ---- debug (remove later) ----
        print("Input mean:", processed.mean())
        print("Input std:", processed.std())

        # ---- predict ----
        pred = model.predict(processed, verbose=0)[0][0]

        print("MRI prediction:", pred)

        # ---- your threshold rule ----
        if pred <= 0.5:
            label = "epilepsy"
        else:
            label = "healthy"

        state.mri_epilepsy_label = label

    except Exception as e:
        print("MRI error:", e)
        state.mri_epilepsy_label = "uncertain"

    return state

In [87]:
if __name__ == "__main__":
    state = EpilepsyState(
        mri_image_path="/home/aasritha/EpilepsyNexus/15029350.png"
    )

    result = mri_classifier_node(state)
    print("MRI result:", result.mri_epilepsy_label)


Input mean: 0.34686768
Input std: 0.20766312
MRI prediction: 0.29469448
MRI result: epilepsy
